In [1]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" trackio \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth trackio
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

In [2]:
%%capture
!pip install -qqq fastapi uvicorn requests
!git clone https://github.com/dpiresearch/OpenEnv.git
%cd OpenEnv
import subprocess, sys, os
from pathlib import Path
sys.path.insert(0, './src')
working_directory = str(Path.cwd().parent.absolute() / "OpenEnv")

In [3]:
!pwd
!ls src/envs
!touch src/envs/__init.py__

/workspace/AIAC/OpenEnv
README.md    atari_env	coding_env  finrl_env  openspiel_env
__init.py__  chat_env	echo_env    git_env    sumo_rl_env


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 768 # Can increase for longer RL output
lora_rank = 4        # Larger rank = smarter, but slower
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b-BF16",
    load_in_4bit = False,
    max_seq_length = max_seq_length,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 10-28 22:21:20 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Gpt_Oss patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 191.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free licens

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [18]:
#
# Make it so we can recognize the new OpenEnv directory holding the new environments
#
%cd OpenEnv
%cd src
!pwd
!ls src/envs
!touch src/envs/__init.py__
from envs.stock_patterns_env import StockPatternEnv
from envs.stock_patterns_env.models import StockPatternAction, StockPatternObservation, StockPatternState
from envs.stock_patterns_env.server.stock_patterns_environment import StockPatternEnvironment


[Errno 2] No such file or directory: 'OpenEnv'
/workspace/AIAC/OpenEnv/OpenEnv/src
[Errno 2] No such file or directory: 'src'
/workspace/AIAC/OpenEnv/OpenEnv/src
/workspace/AIAC/OpenEnv/OpenEnv/src
ls: cannot access 'src/envs': No such file or directory
touch: cannot touch 'src/envs/__init.py__': No such file or directory


In [19]:
import sys
import random
import argparse
from pathlib import Path
from collections import defaultdict


In [20]:

class SimpleRLAgent:
    """
    Ultra-simple Q-learning agent for demonstration.
    
    Uses a simple discretization strategy and tabular Q-learning.
    """
    
    def __init__(self, learning_rate=0.1, discount=0.95, epsilon=0.2):
        self.lr = learning_rate
        self.gamma = discount
        self.epsilon = epsilon
        self.q_values = defaultdict(lambda: [0.0, 0.0, 0.0])  # [Hold, Buy, Sell]
    
    def get_state(self, prices, position):
        """
        Convert observation to simple state representation.
        
        Uses last 3 price movements (up/down) and current position.
        """
        if len(prices) < 4:
            return (0, 0, 0, position)
        
        # Get last 3 price changes
        recent = prices[-4:]
        movements = []
        for i in range(1, 4):
            if recent[i] > recent[i-1]:
                movements.append(1)  # Up
            else:
                movements.append(0)  # Down
        
        return tuple(movements + [position])
    
    def select_action(self, state, position):
        """Select action using epsilon-greedy policy."""
        # Determine valid actions based on position
        if position == 0:
            valid_actions = [0, 1]  # Hold or Buy
        else:
            valid_actions = [0, 2]  # Hold or Sell
        
        # Epsilon-greedy
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        # Choose best Q-value among valid actions
        q_vals = self.q_values[state]
        return max(valid_actions, key=lambda a: q_vals[a])
    
    def learn(self, state, action, reward, next_state, done):
        """Update Q-value using Q-learning rule."""
        current_q = self.q_values[state][action]
        
        if done:
            target_q = reward
        else:
            max_next_q = max(self.q_values[next_state])
            target_q = reward + self.gamma * max_next_q
        
        # Q-learning update
        self.q_values[state][action] += self.lr * (target_q - current_q)


In [21]:
def run_rl_training(episodes=50, max_steps=80, verbose=True):
    """
    Run the RL training loop.
    
    Args:
        episodes: Number of training episodes
        max_steps: Maximum steps per episode
        verbose: Print progress
    """
    print("🎓 Simple RL Training on Stock Patterns")
    print("=" * 60)
    
    # Create environment (local, no server needed)
    env = StockPatternEnvironment(
        pattern_name=None,  # Random patterns
        difficulty=0.4,     # Medium difficulty
    )
    
    # Create agent
    agent = SimpleRLAgent(
        learning_rate=0.1,
        discount=0.95,
        epsilon=0.2,
    )
    
    # Track performance
    all_rewards = []
    all_returns = []
    profitable_episodes = 0
    
    print(f"Training for {episodes} episodes...\n")
    
    # Training loop
    for episode in range(episodes):
        # Reset environment for new episode
        obs = env.reset()
        
        episode_reward = 0
        step = 0
        
        # Get initial state
        state = agent.get_state(obs.prices, obs.position)
        
        # Run episode
        while not obs.done and step < max_steps:
            # Agent selects action
            action = agent.select_action(state, obs.position)
            
            # Execute action in environment
            obs = env.step(StockPatternAction(
                action_type="trade",
                trade_action=action
            ))
            
            # Get reward
            reward = obs.reward or 0.0
            episode_reward += reward
            
            # Get next state
            next_state = agent.get_state(obs.prices, obs.position)
            
            # Learn from experience
            agent.learn(state, action, reward, next_state, obs.done)
            
            # Move to next state
            state = next_state
            step += 1
        
        # Track statistics
        final_return = env.state.total_return
        all_rewards.append(episode_reward)
        all_returns.append(final_return)
        
        if final_return > 0:
            profitable_episodes += 1
        
        # Print progress
        if verbose and (episode + 1) % 10 == 0:
            avg_reward = sum(all_rewards[-10:]) / min(10, len(all_rewards))
            avg_return = sum(all_returns[-10:]) / min(10, len(all_returns))
            win_rate = profitable_episodes / (episode + 1) * 100
            
            print(f"Episode {episode + 1}/{episodes}")
            print(f"  Avg Reward (last 10): {avg_reward:.2f}")
            print(f"  Avg Return (last 10): {avg_return:.2f}%")
            print(f"  Win Rate: {win_rate:.1f}%")
            print(f"  States learned: {len(agent.q_values)}")
            print()
    
    # Final statistics
    print("=" * 60)
    print("✅ Training Complete!")
    print("=" * 60)
    
    avg_reward = sum(all_rewards) / len(all_rewards)
    avg_return = sum(all_returns) / len(all_returns)
    win_rate = profitable_episodes / episodes * 100
    
    print(f"Overall Performance:")
    print(f"  Average Reward: {avg_reward:.2f}")
    print(f"  Average Return: {avg_return:.2f}%")
    print(f"  Win Rate: {win_rate:.1f}% ({profitable_episodes}/{episodes})")
    print(f"  States Learned: {len(agent.q_values)}")
    print()
    
    if avg_return > 0:
        print("💰 Agent learned to make profitable trades on average!")
    else:
        print("📉 Agent needs more training or hyperparameter tuning.")
    
    # Show some learned Q-values
    print("\n📚 Sample Learned Q-Values:")
    print("   State          ->  [Hold,  Buy, Sell]")
    print("-" * 50)
    for i, (state, q_vals) in enumerate(list(agent.q_values.items())[:5]):
        print(f"   {state} -> [{q_vals[0]:6.2f}, {q_vals[1]:6.2f}, {q_vals[2]:6.2f}]")
    print()
    
    return {
        'rewards': all_rewards,
        'returns': all_returns,
        'win_rate': win_rate,
        'agent': agent,
    }


def test_trained_agent(agent, episodes=10):
    """
    Test the trained agent on new episodes.
    
    Args:
        agent: Trained agent
        episodes: Number of test episodes
    """
    print("\n" + "=" * 60)
    print("🎯 Testing Trained Agent")
    print("=" * 60)
    
    env = StockPatternEnvironment(difficulty=0.4)
    
    # Disable exploration for testing
    old_epsilon = agent.epsilon
    agent.epsilon = 0.0
    
    test_returns = []
    wins = 0
    
    for ep in range(episodes):
        obs = env.reset()
        state = agent.get_state(obs.prices, obs.position)
        step = 0
        
        while not obs.done and step < 80:
            action = agent.select_action(state, obs.position)
            obs = env.step(StockPatternAction(action_type="trade", trade_action=action))
            state = agent.get_state(obs.prices, obs.position)
            step += 1
        
        final_return = env.state.total_return
        test_returns.append(final_return)
        
        if final_return > 0:
            wins += 1
        
        pattern_name = env.state.pattern_name
        print(f"  Test {ep+1}: Return={final_return:6.2f}%, Pattern={pattern_name}")
    
    # Restore epsilon
    agent.epsilon = old_epsilon
    
    avg_return = sum(test_returns) / len(test_returns)
    win_rate = wins / episodes * 100
    
    print(f"\nTest Results:")
    print(f"  Average Return: {avg_return:.2f}%")
    print(f"  Win Rate: {win_rate:.1f}%")
    print()


In [22]:
"""Main function."""
"""
parser = argparse.ArgumentParser(
    description='Simple RL training on Stock Patterns'
)
parser.add_argument('--episodes', type=int, default=50,
                   help='Number of training episodes (default: 50)')
parser.add_argument('--test-episodes', type=int, default=10,
                   help='Number of test episodes (default: 10)')
parser.add_argument('--max-steps', type=int, default=80,
                   help='Max steps per episode (default: 80)')
print("Before parsing")
args = parser.parse_args()
print("After parsing")
"""
try:
    # Run training
    '''
    results = run_rl_training(
            episodes=args.episodes,
            max_steps=args.max_steps,
            verbose=True
        )
    '''
    results = run_rl_training(
        episodes=50,
        max_steps=80,
        verbose=True
    )
    
    # Test trained agent
#        test_trained_agent(results['agent'], episodes=args.test_episodes)
    test_trained_agent(results['agent'], episodes=10)
    
    print("=" * 60)
    print("🎉 All Done!")
    print("=" * 60)
    print("\nKey Takeaways:")
    print("• The agent learns Q-values for different price patterns")
    print("• Exploration (epsilon) helps discover good strategies")
    print("• The agent learns to buy low and sell high")
    print("• More episodes → better performance (usually)")
    print("\nNext Steps:")
    print("• Try adjusting learning_rate, discount, epsilon")
    print("• Increase episodes for better convergence")
    print("• Add more features to state representation")
    print("• Implement more sophisticated RL algorithms (DQN, PPO)")
    print()
    
except KeyboardInterrupt:
    print("\n\n⚠️  Training interrupted by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


🎓 Simple RL Training on Stock Patterns
Training for 50 episodes...

Episode 10/50
  Avg Reward (last 10): 2.92
  Avg Return (last 10): 2.74%
  Win Rate: 60.0%
  States learned: 16

Episode 20/50
  Avg Reward (last 10): 8.54
  Avg Return (last 10): 8.89%
  Win Rate: 75.0%
  States learned: 16

Episode 30/50
  Avg Reward (last 10): 9.07
  Avg Return (last 10): 5.98%
  Win Rate: 70.0%
  States learned: 16

Episode 40/50
  Avg Reward (last 10): 12.85
  Avg Return (last 10): 13.61%
  Win Rate: 72.5%
  States learned: 16

Episode 50/50
  Avg Reward (last 10): -0.13
  Avg Return (last 10): -0.85%
  Win Rate: 64.0%
  States learned: 16

✅ Training Complete!
Overall Performance:
  Average Reward: 6.65
  Average Return: 6.07%
  Win Rate: 64.0% (32/50)
  States Learned: 16

💰 Agent learned to make profitable trades on average!

📚 Sample Learned Q-Values:
   State          ->  [Hold,  Buy, Sell]
--------------------------------------------------
   (0, 1, 1, 0) -> [  2.09,   3.85,   0.00]
   (1, 1